In [1]:
import pandas as pd
import numpy as np
import os
import glob


In [2]:
def fr_to_float(series):
    series = series.astype(str)
    series = series.str.replace("\xa0", "", regex=False)  # NBSP
    series = series.str.replace(".", "", regex=False)      # thousands sep
    series = series.str.replace(",", ".", regex=False)     # decimal comma
    series = series.str.replace("-", "", regex=False)      # empty cells
    series = series.str.strip()
    return pd.to_numeric(series, errors="coerce")


In [3]:
def parse_volume(series):
    s = series.astype(str).str.strip()
    mult = s.str.extract(r'([KM])$', expand=False)
    s = s.str.replace(r'[KM]', '', regex=True).str.strip()
    s = fr_to_float(s)
    factor = mult.map({'K': 1e3, 'M': 1e6})
    return s * factor.fillna(1)


In [4]:
def clean_market_file(path):
    df = pd.read_csv(path)

    # Normalize column names
    df.columns = (
        df.columns.astype(str)
            .str.replace("\xa0", " ", regex=False)
            .str.replace(r"\s+", " ", regex=True)
            .str.strip()
    )

    col_map = {
        "Date": "date",
        "Dernier": "close",
        "Ouv.": "open",
        "Plus Haut": "high",
        "Plus Bas": "low",
        "Vol.": "volume",
        "Variation %": "variation_pct"
    }

    df = df.rename(columns=col_map)

    # Clean price columns
    for col in ["close", "open", "high", "low"]:
        df[col] = fr_to_float(df[col])

    # Clean volume
    df["volume"] = parse_volume(df["volume"])

    # Clean variation %
    df["variation_pct"] = (
        df["variation_pct"]
            .astype(str)
            .str.replace("%", "", regex=False)
            .str.strip()
    )
    df["variation_pct"] = fr_to_float(df["variation_pct"]) / 100

    # Convert date
    df["date"] = pd.to_datetime(df["date"], dayfirst=True, errors="coerce")
    df = df.sort_values("date").reset_index(drop=True)

    # Extract ticker from filename
    base = os.path.basename(path)
    first = base.split(" ")[0]

    if first == "CAC":
        ticker = "CAC40"
    else:
        ticker = first

    df["ticker"] = ticker

    return df


In [5]:
input_dir = "stocks"
output_dir = "cleaned_stocks"

os.makedirs(output_dir, exist_ok=True)

# Get all CSVs in the folder
files = glob.glob(os.path.join(input_dir, "*.csv"))

print("Found", len(files), "raw files.")

# Clean each file
for path in files:
    print("Processing:", os.path.basename(path))

    df_clean = clean_market_file(path)
    ticker = df_clean["ticker"].iloc[0]

    output_path = os.path.join(output_dir, f"{ticker}_clean.csv")
    df_clean.to_csv(output_path, index=False)

    print("  Saved:", output_path)

print("\nAll files cleaned successfully!")
print(os.listdir(output_dir))


Found 41 raw files.
Processing: ACCP - Données Historiques.csv
  Saved: cleaned_stocks\ACCP_clean.csv
Processing: AIR - Données Historiques.csv
  Saved: cleaned_stocks\AIR_clean.csv
Processing: AIRP - Données Historiques .csv
  Saved: cleaned_stocks\AIRP_clean.csv
Processing: AXAF - Données Historiques.csv
  Saved: cleaned_stocks\AXAF_clean.csv
Processing: BNPP - Données Historiques.csv
  Saved: cleaned_stocks\BNPP_clean.csv
Processing: BOUY - Données Historiques.csv
  Saved: cleaned_stocks\BOUY_clean.csv
Processing: BVI - Données Historiques.csv
  Saved: cleaned_stocks\BVI_clean.csv
Processing: CAC40 - Données Historiques.csv
  Saved: cleaned_stocks\CAC40_clean.csv
Processing: CAGR - Données Historiques.csv
  Saved: cleaned_stocks\CAGR_clean.csv
Processing: CAPP - Données Historiques.csv
  Saved: cleaned_stocks\CAPP_clean.csv
Processing: CARR - Données Historiques.csv
  Saved: cleaned_stocks\CARR_clean.csv
Processing: DANO - Données Historiques.csv
  Saved: cleaned_stocks\DANO_clean.c